In [1]:
from flask import Flask, request, jsonify
from waitress import serve
from mysql.connector import connect
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from flask_cors import CORS, cross_origin

import smtplib, ssl
import openai
import re

In [2]:
env = 'development'
# env = 'production'

In [3]:
# DB Parameters
if env == 'production':
    user = ''
    password = ''
    host = ''
    database = ''
else:
    user = 'root'
    password = ''
    host = 'localhost'
    database = 'chatgpt_recommendations'

In [4]:
# SMTP Parametrs
if env == 'production':
    smtp_port = ""
    smtp_server = ""
    sender_email = ""
    smtp_password = ""
else:
    smtp_port = 465
    smtp_server = "smtp.gmail.com"
    sender_email = ""
    smtp_password = ""
context = ssl.create_default_context()

In [5]:
# CORS Prameters
if env == 'production':
    cors_allowed_urls = ["https://www.example.com"]
else:
    cors_allowed_urls = ["http://localhost"]

In [6]:
# OPENAI KEY
if env == 'production':
    api_key=""
else:
    api_key=""

In [7]:
app = Flask(__name__)
cors = CORS(app)
con = connect(host=host, database=database, user=user, password=password)

In [8]:
# Create info table
create_table = con.cursor()
create_table.execute('''
    CREATE TABLE IF NOT EXISTS `info` (
      `id` int(11) NOT NULL AUTO_INCREMENT,
      `query` varchar(255) DEFAULT NULL,
      `email` varchar(255) DEFAULT NULL,
      `query_id` varchar(255) DEFAULT NULL,
      `chatbot_reply` text DEFAULT NULL,
      `created_at` datetime DEFAULT current_timestamp(),
      PRIMARY KEY (`id`)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_general_ci
''')

In [9]:
def extractUserInfo(data):
    query = data['query']
    email = data['email']
    queryId = data['queryId']
    return query, email, queryId

In [10]:
def getChatbotReply(query):
    try:
        client = openai.OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": query}]
        )
        return True, response.choices[0].message.content.replace("\n", "<br/>")

    except openai.APIError as e:
      #Handle API error here, e.g. retry or log
      return False, "OpenAI API returned an API Error: " + e.message
    except openai.APIConnectionError as e:
      #Handle connection error here
      return False, "Failed to connect to OpenAI API: " + e.message
    except openai.RateLimitError as e:
      #Handle rate limit error (we recommend using exponential backoff)
      return False, "OpenAI API request exceeded rate limit: " + e.message
    except:
        return False, 'Error during get response from chatbot.'

In [11]:
def save_info(query, email, queryId, chatbot_reply):
    cursor = con.cursor()
    cursor.execute(("INSERT INTO info (query, email, query_id, chatbot_reply) VALUE (%s, %s, %s, %s)"), (query, email, queryId, chatbot_reply))
    con.commit()

In [12]:
def validate_email(email):
    if re.match(r"[^@]+@[^@]+\.[^@]+", email):
        return True
    return False

In [13]:
def prepare_message(recommendations_list_from_chatbot, receiver_email):
    message = MIMEMultipart("alternative")
    message["Subject"] = "ChatGPT Recommendations"
    message["From"] = sender_email
    message["To"] = receiver_email

    html = """\
    <html>
      <body>
        <p>Thanks for using our bot.</p>
        <p>Below are your recommended ideas and we hope the one(s) you choose bring joy and happiness to your loved one.</p>
        <p>If you enjoyed this experience, please tell a friend about this <strong>Bot</strong>!</p>
        <p>"""+recommendations_list_from_chatbot+"""</p>
        <p>Happy Recommendations,</p>
        <p>ChatGPT</p>
      </body>
    </html>
    """

    message.attach(MIMEText(html, "html"))
    
    return message.as_string()

In [14]:
def notify_user(recommendations_list_from_chatbot, receiver_email):
    if(validate_email(receiver_email)):
        message = prepare_message(recommendations_list_from_chatbot, receiver_email)

        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
            server.login(sender_email, smtp_password)
            server.sendmail(sender_email, receiver_email, message)

In [15]:
@app.route("/", methods=['POST'])
@cross_origin(cors_allowed_urls)
def index():
    query, email, queryId = extractUserInfo(request.get_json())
    isChatBotSuccessfully, chatbot_reply = getChatbotReply(query)
    save_info(query, email, queryId, chatbot_reply)
    if isChatBotSuccessfully:
        notify_user(chatbot_reply, email)
        return jsonify({'error': 0, 'message': 'Thanks, the bot is researching the best recommendations for your loved one. You can check your email for the results.'})
    else:
        return jsonify({'error': 1, 'message': 'Uh oh, we ran into an error but our engineers are working on the issue. Please try again in 5 minutes.'})

In [ ]:
if __name__ == '__main__':
    #serve(app, host="0.0.0.0", port=8080)
    app.run()
    con.close()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2023 19:35:15] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:35:27] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:35:30] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:35:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:40:19] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:40:42] "POST / HTTP/1.1" 200 -
